In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import requests
import json

In [2]:
'''
strona: https://www.cars-data.com/en/all-cars/page1.html
robots.txt:
User-agent: *
Allow: /

Sitemap: https://www.cars-data.com/en/val-en.xml
Sitemap: https://www.cars-data.com/en/en-index.xml	
Sitemap: https://www.cars-data.com/fr/fr-index.xml	
Sitemap: https://www.cars-data.com/de/de-index.xml	
Sitemap: https://www.cars-data.com/no/no-index.xml	
Sitemap: https://www.cars-data.com/es/es-index.xml	
Sitemap: https://www.cars-data.com/cz/cz-index.xml	
Sitemap: https://www.cars-data.com/hu/hu-index.xml	
Sitemap: https://www.cars-data.com/ro/ro-index.xml	
Sitemap: https://www.cars-data.com/srb/srb-index.xml	
Sitemap: https://www.cars-data.com/jp/jp-index.xml	
Sitemap: https://www.cars-data.com/hi/hi-index.xml	
Sitemap: https://www.cars-data.com/za/za-index.xml	
Sitemap: https://www.cars-data.com/se/se-index.xml
Sitemap: https://www.cars-data.com/el/el-index.xml

User-agent: *
Disallow: /cn/
'''

'\nstrona: https://www.cars-data.com/en/all-cars/page1.html\nrobots.txt:\nUser-agent: *\nAllow: /\n\nSitemap: https://www.cars-data.com/en/val-en.xml\nSitemap: https://www.cars-data.com/en/en-index.xml\t\nSitemap: https://www.cars-data.com/fr/fr-index.xml\t\nSitemap: https://www.cars-data.com/de/de-index.xml\t\nSitemap: https://www.cars-data.com/no/no-index.xml\t\nSitemap: https://www.cars-data.com/es/es-index.xml\t\nSitemap: https://www.cars-data.com/cz/cz-index.xml\t\nSitemap: https://www.cars-data.com/hu/hu-index.xml\t\nSitemap: https://www.cars-data.com/ro/ro-index.xml\t\nSitemap: https://www.cars-data.com/srb/srb-index.xml\t\nSitemap: https://www.cars-data.com/jp/jp-index.xml\t\nSitemap: https://www.cars-data.com/hi/hi-index.xml\t\nSitemap: https://www.cars-data.com/za/za-index.xml\t\nSitemap: https://www.cars-data.com/se/se-index.xml\nSitemap: https://www.cars-data.com/el/el-index.xml\n\nUser-agent: *\nDisallow: /cn/\n'

In [1]:
chrome_options = Options()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=chrome_options)
driver.get(f'https://www.cars-data.com/en/all-cars/page1.html')
WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p'))).click()
id = 0

for num in range(1,98,1):
    driver1 = webdriver.Chrome(options=chrome_options)
    driver1.get('https://www.cars-data.com/en/all-cars/page'+str(num)+'.html')
    WebDriverWait(driver1, 1).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p'))).click()
    section = driver1.find_element(By.XPATH, "/html/body/main/div/div[2]/section")
    classes = section.find_elements(By.CLASS_NAME, "col-4")
    
    key = ''
    for link in classes:
        href = link.find_element(By.TAG_NAME, "a")
        driver2 = webdriver.Chrome(options=chrome_options)
        driver2.get(href.get_attribute("href"))
        title = driver2.find_element(By.XPATH, "/html/body/main/div/div[2]/div[1]/section/div[2]/h1")
        years = link.find_element(By.TAG_NAME, "p")
        dictionary = {'id':id, 'model':title.text, 'years':years.text}
        
        WebDriverWait(driver2, 1).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p'))).click()
        
        tds = driver2.find_elements(By.TAG_NAME, "td")

        i = 0
        for td in tds:
            if i%2==1: #key
                key = td.text
            else:
                dictionary[key] = td.text

            i += 1

        del dictionary['']
        driver2.quit()
        data = json.dumps(dictionary)
        with open('jsons/data_{0}_{1}.json'.format(num, id), 'w') as f:
            json.dump(dictionary, f, indent=4)
        id += 1
    print(num, id)
    
    driver1.quit()

driver.quit()

In [46]:
import os
import pandas as pd

directory = os.fsencode('jsons')

with open('jsons/data_1_0.json') as json_file:
    data = json.load(json_file)
    df1 = pd.DataFrame([data], index=[data['id']])

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".json") and filename != 'data_1_0.json': 
        with open('jsons/'+filename) as json_file:
            data = json.load(json_file)
            df2 = pd.DataFrame([data])
            df1 = pd.concat([df1, df2])
print(df1)

      id                                             model  \
431  431  Citroen C3 Pluriel 1.4 2003 Manual 2 doors specs   

                                 years search    Price: New Price Roadworthy:  \
431  2003 - 2010\n2 doors, Convertible         € 20.890                   n/a   

    Road Tax / 3 Months:            Body Type:                Transmission:  \
431        € 115 - € 128  2-doors, convertible  5 speed manual transmission   

    Drive Wheel :  ... New Price 2009: New Price 2008: New Price 2007:  \
431         front  ...        € 20.890        € 19.675        € 19.675   

    New Price 2006: Occasion Price 2010: 4 Years Depreciation:  \
431        € 18.990              € 3.100                 € 250   

    Motor Vehicle Tax: Insurance: Fuel Costs: Maintenance:  
431               € 38       € 47       € 173         € 46  

[1 rows x 93 columns]


In [55]:
df1.to_csv('cars.csv', sep=';')